In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [38]:
final_data = pd.read_csv('/Users/surajmathapati/Prep/Customer_churn_prediction/notebooks/final_data.csv')
final_data.head()

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,gender_Male
0,15634602,619,42,2,0.00,1,1,1,101348.88,1,False,False
1,15619304,502,42,8,159660.80,3,1,0,113931.57,1,False,False
2,15701354,699,39,1,0.00,2,0,0,93826.63,0,False,False
3,15592531,822,50,7,0.00,2,1,1,10062.80,0,False,True
4,15656148,376,29,4,115046.74,4,1,0,119346.88,1,True,False


In [39]:
X = final_data.drop(columns=['churn', 'customer_id'])
y = final_data['churn']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:

X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

In [42]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [43]:
y_pred = model.predict(X_test)

In [44]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("Classification Report:\n", classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.7102
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.72      0.80      1770
           1       0.40      0.68      0.50       487

    accuracy                           0.71      2257
   macro avg       0.65      0.70      0.65      2257
weighted avg       0.78      0.71      0.73      2257

Confusion Matrix:
 [[1272  498]
 [ 156  331]]


In [45]:
import joblib
joblib.dump(model, "../models/logistic_regression_model.pkl")
joblib.dump(scaler, "../models/scaler.pkl")


['../models/scaler.pkl']

In [46]:
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Class distribution after SMOTE:")
print(y_train_smote.value_counts()) 

Class distribution after SMOTE:
churn
0    4719
1    4719
Name: count, dtype: int64


In [50]:
model = LogisticRegression(max_iter=500, solver='lbfgs')
model.fit(X_train_smote, y_train_smote)

y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7216
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.77      0.81      1180
           1       0.40      0.56      0.46       325

    accuracy                           0.72      1505
   macro avg       0.63      0.66      0.64      1505
weighted avg       0.76      0.72      0.74      1505

Confusion Matrix:
 [[905 275]
 [144 181]]


/Users/surajmathapati/.pyenv/versions/3.13.0b4/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
